In [1]:
import pandas as pd
import re
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
df = pd.read_csv('/content/Sentiment_IDX.csv')
print(df)

                   created_at            author_id  \
0   2025-08-15 09:11:33+00:00           1429796232   
1   2025-08-15 09:06:35+00:00             67387899   
2   2025-08-15 09:06:30+00:00  1895255815856234496   
3   2025-08-15 09:05:17+00:00  1849435909185490944   
4   2025-08-15 09:02:17+00:00  1323800017530875904   
5   2025-08-15 08:41:18+00:00            165385438   
6   2025-08-15 08:41:05+00:00  1139602866576842752   
7   2025-08-15 08:39:49+00:00  1331503267814072320   
8   2025-08-15 08:39:12+00:00  1767713020061556736   
9   2025-08-15 08:38:35+00:00   847372542830444544   
10  2025-08-15 08:37:22+00:00            311380271   
11  2025-08-15 08:36:28+00:00            577790231   
12  2025-08-15 08:35:15+00:00  1918670567143776256   
13  2025-08-15 08:32:26+00:00  1379125037878288389   
14  2025-08-15 08:31:33+00:00   847372542830444544   
15  2025-08-15 08:30:57+00:00  1737826637251764224   
16  2025-08-15 08:27:51+00:00  1927285286871924736   
17  2025-08-15 08:22:09+00:0

In [3]:
def clean_text(text):
  if isinstance(text, float):
    text = str(text)

  text = text.lower()
  text = re.sub(r'http\S+', '', text)                   # Hapus URL
  text = re.sub(r'@\w+', '', text)                      # Hapus mention
  text = re.sub(r'#\w+', '', text)                      # Hapus hashtag
  text = re.sub(r'[^\w\s]', '', text)                   # Hapus tanda baca
  text = re.sub(r'\s+', ' ', text).strip()              # Hapus spasi berlebih
  stop_words = set(stopwords.words('indonesian'))
  words = text.split()
  words = [word for word in words if word not in stop_words and len(word) > 2]
  return ' '.join(words)

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax

model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def predict_sentiment(text):
  cleaned_text = clean_text(text)
  inputs = tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
  with torch.no_grad():
    outputs = model(**inputs)
    predictions = softmax(outputs.logits, dim=-1)

  predicted_class = torch.argmax(predictions, dim=-1).item()
  confidence = torch.max(predictions, dim=-1)[0].item()
  label_mapping = {0: "negative", 1: "neutral", 2: "positive"}
  sentiment_label = label_mapping.get(predicted_class, "Unknown")

  return sentiment_label, predicted_class, confidence, predictions.numpy()[0]

                                                 text  \
0   IHSG ATH 8000 cuman beberapa menit doang, gk a...   
1   sempat lanjut bergerak naik di sesi perdaganga...   
2   beneran ini keknya target $IHSG 8000 HUT RI 80...   
3   IHSG 8k nggak sampai semenit, trus 17 Agustus ...   
4   Prabowo menilai kinerja saham Indonesia menunj...   
5           @Stockbit IHSG juga bisa dipesan ternyata   
6   IHSG 8000 langsung di koreksi aowkwkwk kasian ...   
7   Breaking News \n\nIHSG mecetak rekor tertinggi...   
8   @BigAlphaID @prabowo Mana para pakar IHSG yg k...   
9   Setelah Sentuh 8.000, IHSG Sesi I Koreksi Tipi...   
10  Emang tema pemerintahan kali ini adalah "measu...   
11  Halah org cuma bertahan itungan menit doang\nT...   
12  Jelang HUT KEMERDEKAAN RI IHSG jadi JUARA, mas...   
13  $IHSG \nKoreksi dulu boleh dong, kan ga harus ...   
14  IHSG Tembus 8.000, Ditopang Saham Konglo Hingg...   
15  Besok caranya ga begini mainya saham kebanyaka...   
16  Chandra Widodo: Likuiditas 